In [1]:
#This line has been inserted as Github commit test

def qld_data(datetime,requests,ET,pd):
    
    now = datetime.datetime.now()
    now = now.strftime('%Y%m%d-%H%M%S')
    
    qld_fire_data = 'data/qld_data/qld_fire_data_'+now+'.xml'
    
        #add the data from the website 
    URL = "https://www.qfes.qld.gov.au/data/alerts/bushfireAlert.xml"
    
    #The binary mode (wb) is required to write the actual content of the XML page to your external file in the original format.
    #will get the current up to date details from website
    response = requests.get(URL)
    with open(qld_fire_data, 'wb') as file:
        file.write(response.content)
        
    #parse through the data    
    mytree = ET.parse(qld_fire_data) 
    myroot = mytree.getroot()

    geo_list=[]
    for geo in myroot.findall('.//{http://www.georss.org/georss}point'):
        geo_list.append(geo.text)

    co_ordinates_split =[]
    for value in geo_list:
        co_ordinates_split.append(value.split())

    qld_latitude_list = []
    qld_longitude_list = []
    for co_ordinates in co_ordinates_split:
        qld_latitude_list.append(co_ordinates[0])
        qld_longitude_list.append(co_ordinates[1])
    
    report_date = datetime.date.today()
    report_date = report_date.strftime('%Y-%m-%d')
    
    qld_df = pd.DataFrame({
        'Latitude': qld_latitude_list,
        'Longitude': qld_longitude_list,
        'State': 'QLD',
        'Report Date': report_date
    })
    
    #Do we need these steps for our end code or should we go straight to S3 rather than saving it down
    qld_df.to_csv("data/fire_data.csv", mode = 'a', index = False , header = False)

    #remove duplicate values
    fire_data_df = pd.read_csv('data/fire_data.csv')

    #drop the duplicates out of the data frame
    distinct_fire_data = fire_data_df.drop_duplicates()

    #create a new csv file with the dropped duplicate values
    distinct_fire_data.to_csv('data/fire_data_distinct.csv',index = False)
